In [1]:
import random
import numpy as np

In [2]:
# 미로 찾기 : 에이전트가 움직이면서 목표 지점 (4, 6)에 도달하고자 함
class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0

    def step(self, a): # 새로운 액션 a 수행
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1 # 보상은 항상 -1로 고정 -> 최대한 빠르게 목표 도달 목적
        done = self.is_done() # done : 목표 도달 여부( True / False )
        return (self.x, self.y), reward, done # (새로운 x, 새로운 y), 보상, 목표 도달 여부

    def move_left(self): # 0번 액션 : 왼쪽 방향
        if self.y==0: #( . 0) 이동 금지
            pass
        elif self.y==3 and self.x in [0,1,2]: #(0~2, 3) 이동 금지
            pass
        elif self.y==5 and self.x in [2,3,4]: #(2~4, 5) 이동 금지
            pass
        else:
            self.y -= 1 # 나머지 경우는 모두 이동 가능

    def move_right(self): # 2번 액션 : 오른쪽 방향
        if self.y==1 and self.x in [0,1,2]:
            pass
        elif self.y==3 and self.x in [2,3,4]:
            pass
        elif self.y==6:
            pass
        else:
            self.y += 1

    def move_up(self): # 1번 액션 : 위쪽 방향
        if self.x==0:
            pass
        elif self.x==3 and self.y==2:
            pass
        else:
            self.x -= 1

    def move_down(self): # 3번 액션 : 아래쪽 방향
        if self.x==4:
            pass
        elif self.x==1 and self.y==4:
            pass
        else:
            self.x+=1

    def is_done(self):
        if self.x==4 and self.y==6: # 현재 위치가 (4, 6) 경우 목표 도달
            return True # True
        else:
            return False

    def reset(self): # 초기화 부분
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [3]:
# GridWorld에서 Q-learning으로 최적의 정책 학습
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # Q 테이블을 0으로 초기화
                                           # Q-table : (행 위치 x, 열 위치 y, 행동 a)
        self.eps = 0.9 # 입실론값 초기값 : 0.9 -> 90% 확률로 탐험 / 10% 확률로 greedy 이용

    # 입실론- greedy 정책으로 액션 선택
    def select_action(self, s):
        # eps-greedy로 액션을 선택해준다
        x, y = s # s : 현재 상태
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3) # 랜덤으로 약션 탐험
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    # Q-learning으로 테이블 업데이트
    def update_table(self, transition):
        s, a, r, s_prime = transition
        x,y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) # S'에서 선택할 액션 (실제로 취한 액션이 아님)
        # Q러닝 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1 * (r + np.amax(self.q_table[next_x,next_y,:]) - self.q_table[x,y,a]) # a : 학습률

    # 탐험 -> 이용
    def anneal_eps(self):
        self.eps -= 0.01  # Q러닝에선 epsilon 이 좀더 천천히 줄어 들도록 함. -> 탐험에서 이용 중심으로
        self.eps = max(self.eps, 0.2)

    # 최종 학습 결과 출력
    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)): # 각 위치에서 가장 Q값이 큰 액션 출력
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [4]:
# 메인 루프
def main():
    env = GridWorld() # GridWorld 환경 초기화
    agent = QAgent() # QAgent 즉 에이전트 초기화

    # 에피소드 반복
    for n_epi in range(1000): # 에피소드 1000개 반복 (각각 (0, 0) -> (4, 6))
        done = False

        # 환경 초기화 & 상태 리셋
        s = env.reset()
        # done이 아님 => 목표 상태 도달 X
        while not done:
            a = agent.select_action(s) # 액션 선택
            s_prime, r, done = env.step(a) # 액션 수행하고 다음 상태, 보상, 목표 도달 여부
            agent.update_table((s,a,r,s_prime)) # Q-table 업데이트
            s = s_prime
        agent.anneal_eps() # 탐험 줄임 -> 점점 학습된 정책 따르도록 함

    agent.show_table()

In [5]:
if __name__ == '__main__':
    main()

[[2. 3. 0. 2. 3. 3. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [3. 1. 1. 1. 0. 2. 0.]]
